<a href="https://colab.research.google.com/github/emreakbulut06/Yapay_Sinir_Aglari_Vize_Odevi/blob/main/Yapay_Sinir_Aglari_Vize_Odevi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphanelerin yüklenmesi
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LogisticRegression

# Veri Seti Oluşturma (Simüle Edilmiş Veri)
# Not: Gerçek bir trafik işareti veri seti yerine, rastgele bir veri seti kullanılmıştır.
np.random.seed(42)
num_classes = 43  # Trafik işaret sınıflarının sayısı
num_features = 30  # Özellik sayısı
num_samples = 1000  # Örnek sayısı

# Özellik sütunlarını ve sınıf etiketlerini içeren veri çerçevesi oluşturma
data = pd.DataFrame(
    np.random.rand(num_samples, num_features),
    columns=[f"feature_{i}" for i in range(num_features)]
)
data["label"] = np.random.randint(0, num_classes, size=num_samples)

# Eksik Değerlerin İşlenmesi
# Veri setinde eksik değerler varsa, bu kodlar ile doldurulabilir:
data.fillna(data.mean(), inplace=True)

# Eğitim ve Test Verisi Ayrımı
# Eğitim ve test verisi %80 eğitim, %20 test oranında ayrılmıştır.
X = data.drop(columns=["label"])  # Özellikler
y = data["label"]  # Sınıf etiketleri
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Özellik Ölçekleme
# Verilerin daha iyi performans göstermesi için standartlaştırma işlemi yapılmıştır.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Çok Katmanlı Algılayıcı (MLP) Modeli
mlp_results = []
mlp_parameters = [
    {"hidden_layer_sizes": (50,), "activation": "relu", "learning_rate_init": 0.01},
    {"hidden_layer_sizes": (100,), "activation": "tanh", "learning_rate_init": 0.01},
    {"hidden_layer_sizes": (50, 50), "activation": "relu", "learning_rate_init": 0.001},
]

for params in mlp_parameters:
    # MLP modelinin oluşturulması
    mlp_model = MLPClassifier(
        hidden_layer_sizes=params["hidden_layer_sizes"],
        activation=params["activation"],
        learning_rate_init=params["learning_rate_init"],
        max_iter=500,
        random_state=42
    )
    mlp_model.fit(X_train_scaled, y_train)  # Modeli eğit
    y_pred = mlp_model.predict(X_test_scaled)  # Test veri setiyle tahmin yap
    accuracy = accuracy_score(y_test, y_pred)  # Doğruluk oranını hesapla
    mlp_results.append({"params": params, "accuracy": accuracy})

# MLP sonuçlarını göster
print("MLP Model Results:")
for result in mlp_results:
    print(result)

# Radial Basis Function (RBF) Modeli
class RBFClassifier:
    def __init__(self, gamma=1.0):
        # Gamma, RBF çekirdeğinin duyarlılığını kontrol eder
        self.gamma = gamma
        self.lr_model = LogisticRegression(max_iter=500)

    def fit(self, X, y):
        # RBF çekirdeğini kullanarak özellik dönüşümü yap
        self.X_fit_ = X
        self.y_fit_ = y
        self.rbf_features_ = rbf_kernel(X, X, gamma=self.gamma)
        self.lr_model.fit(self.rbf_features_, y)  # Lojistik regresyon modelini eğit

    def predict(self, X):
        # Test verisi için RBF özellik dönüşümünü uygula ve tahmin yap
        rbf_features = rbf_kernel(X, self.X_fit_, gamma=self.gamma)
        return self.lr_model.predict(rbf_features)

# RBF Modelini Eğit ve Test Et
rbf_model = RBFClassifier(gamma=0.1)
rbf_model.fit(X_train_scaled, y_train)  # Modeli eğit
rbf_predictions = rbf_model.predict(X_test_scaled)  # Test veri setiyle tahmin yap
rbf_accuracy = accuracy_score(y_test, rbf_predictions)  # Doğruluk oranını hesapla

# RBF sonuçlarını göster
print("\nRBF Model Results:")
print({"Gamma": 0.1, "Accuracy": rbf_accuracy})